In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import gc

def getHeatMap(newdataf,metdataf,month,dirpath,count,startDate):
    #Filtraggio per mese
    newdf=newdataf[newdataf['hours'].str.contains(month)]
    metdf=metdataf[metdataf['hours'].str.contains(month)]
    #display(newdf)
    #display(metdf)
    #Creazione dataframe finale
    finaldf=pd.DataFrame(columns=['hours','bool','power_dev','power_met'])
    
    finaldf['hours']=newdf['hours']
    finaldf['bool'] = np.where(newdf.power <= metdataf.power,1,0).astype(int)
    finaldf['power_met']=metdf['power']
    finaldf['power_dev']=newdf['power']
    finaldf.to_csv('finalcsv',index=False)
    writedf=finaldf[finaldf['bool']==0]
    heatDir=os.path.join(os.path.abspath('..'),os.path.join("check\\","EB"+dirpath[17]))
    if not os.path.exists(heatDir):
        os.makedirs(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.csv')
    print(heatmapPath)
    writedf.to_csv(heatmapPath,index=False)
    
    print('HO FINITO')
   
 


    

    
#Funzione che ritorna dataframe in base al mese   
def switch_demo(boolval):
    global jandf
    global metjandf
    if(boolval==False):
        return jandf
    else:
        return metjandf
#Funzione che assegna le modifiche al dataframe in base al mese
def switch_df(df,boolval):
    global jandf
    global metjandf
    if(boolval==False):
        jandf=df
    else:
        metjandf=df
def process(df,filepath,count,boolval):
    global jandf
    global metjandf
    
    
    df['timestamp'] = pd.to_datetime(df['timestamp'],utc=True)
    
    #df['timestamp']=pd.Timestamp(df['timestamp'],tz='Europe/Rome')
    # crea una time series settando la data come index
    #df = df.set_index('timestamp')
    #Aggiungi una colonna contenente  data e ora per heatmap
    df['hours']=df['timestamp'].dt.strftime("%d-%b-%Y (%H-%M-%S)")
    #df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")
    df['day']=df['timestamp'].dt.strftime("%d").str.lstrip("0").replace(" 0", " ")
    df=df.set_index('timestamp')
    display(df)
    monthList=['Mar']
    #Esegui i vari filtraggi e crea un dataframe contenente i valori per ogni mese
    for month in monthList:
        newdf=df[df['hours'].str.contains(month)]
        filterdf=switch_demo(boolval)
        filterdf.loc[filterdf.hours.isin(newdf.hours) , 'power'] = filterdf['power']+newdf['power']
        switch_df(filterdf,boolval)
    del df,newdf,filterdf
    gc.collect()
        
        
        
    
 


In [ ]:
#DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019","\Dati puntuali EB3 Gennaio-Giugno 2019","\Dati puntuali EB4 Gennaio-Giugno 2019","\Dati puntuali EB5 Gennaio-Giugno 2019","\Dati puntuali EB6 Gennaio-Giugno 2019","\Dati puntuali EB7 Gennaio-Giugno 2019","\Dati puntuali EB8 Gennaio-Giugno 2019","\Dati puntuali EB9 Gennaio-Giugno 2019"]
DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019"]
count=0
for dirpath in DIRPATH:
    count=count+1
    print("Start "+str(count))
    dirp=os.path.join(os.path.abspath('..'),"data_rec"+dirpath+"\\")
    filelist=[f for f in os.listdir(dirp)]
    jandf = metjandf= pd.DataFrame({
        'date': pd.date_range(
            start=pd.Timestamp('2019-03'),
            end=pd.Timestamp('2019-03') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
            freq='S',  # <--- try '3h', '6h', '12h' if you want
            closed='left',
            tz='Europe/Rome'
                )
        })
    pd.set_option('display.max_rows', 10000)
    #display(jandf)
    
    jandf = jandf.set_index('date')
    metjandf = metjandf.set_index('date')
    
    jandf['hours']=jandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")
    
    metjandf['hours']=metjandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")

    jandf['power']=metjandf['power']=0
    for filepath in filelist:
        dataPath=os.path.join(dirp,filepath)
        print(filepath)
        if("Meter" in filepath):
            boolval=True
        else:
            boolval=False
        for chunck in pd.read_csv(dataPath, sep=',',chunksize=500000):
            process(chunck,filepath,count,boolval)
            print('Fine')
            print('-----')
    
    #Creazione heatmap
    getHeatMap(jandf,metjandf,'Mar',dirpath,count,'2019-03') 
    
    
    
        

Start 1
Dati puntuali Frigo EB1 2019.csv


,power,hours,day
timestamp,,,
2019-02-28 23:00:00+00:00,34.4,28-Feb-2019 (23-00-00),28
2019-02-28 23:00:01+00:00,34.4,28-Feb-2019 (23-00-01),28
2019-02-28 23:00:02+00:00,34.4,28-Feb-2019 (23-00-02),28
2019-02-28 23:00:03+00:00,34.4,28-Feb-2019 (23-00-03),28
2019-02-28 23:00:04+00:00,34.4,28-Feb-2019 (23-00-04),28
...,...,...,...
2019-03-06 17:53:15+00:00,147.6,06-Mar-2019 (17-53-15),6
2019-03-06 17:53:16+00:00,147.6,06-Mar-2019 (17-53-16),6
2019-03-06 17:53:17+00:00,147.6,06-Mar-2019 (17-53-17),6


Fine
-----


,power,hours,day
timestamp,,,
2019-03-06 17:53:20+00:00,147.6,06-Mar-2019 (17-53-20),6
2019-03-06 17:53:21+00:00,147.6,06-Mar-2019 (17-53-21),6
2019-03-06 17:53:22+00:00,147.6,06-Mar-2019 (17-53-22),6
2019-03-06 17:53:23+00:00,147.6,06-Mar-2019 (17-53-23),6
2019-03-06 17:53:24+00:00,147.6,06-Mar-2019 (17-53-24),6
...,...,...,...
2019-03-12 12:46:35+00:00,36.6,12-Mar-2019 (12-46-35),12
2019-03-12 12:46:36+00:00,36.6,12-Mar-2019 (12-46-36),12
2019-03-12 12:46:37+00:00,36.6,12-Mar-2019 (12-46-37),12


Fine
-----


...

Fine
-----


,power,hours,day
timestamp,,,
2019-03-24 02:33:20+00:00,1.3,24-Mar-2019 (02-33-20),24
2019-03-24 02:33:21+00:00,1.3,24-Mar-2019 (02-33-21),24
2019-03-24 02:33:22+00:00,1.3,24-Mar-2019 (02-33-22),24
2019-03-24 02:33:23+00:00,1.3,24-Mar-2019 (02-33-23),24
2019-03-24 02:33:24+00:00,1.3,24-Mar-2019 (02-33-24),24
...,...,...,...
2019-03-29 21:26:35+00:00,60.3,29-Mar-2019 (21-26-35),29
2019-03-29 21:26:36+00:00,60.3,29-Mar-2019 (21-26-36),29
2019-03-29 21:26:37+00:00,60.3,29-Mar-2019 (21-26-37),29


Fine
-----


,power,hours,day
timestamp,,,
2019-03-29 21:26:40+00:00,60.3,29-Mar-2019 (21-26-40),29
2019-03-29 21:26:41+00:00,60.3,29-Mar-2019 (21-26-41),29
2019-03-29 21:26:42+00:00,60.3,29-Mar-2019 (21-26-42),29
2019-03-29 21:26:43+00:00,60.3,29-Mar-2019 (21-26-43),29
2019-03-29 21:26:44+00:00,60.3,29-Mar-2019 (21-26-44),29
...,...,...,...
2019-03-31 21:59:55+00:00,2.8,31-Mar-2019 (21-59-55),31
2019-03-31 21:59:56+00:00,2.8,31-Mar-2019 (21-59-56),31
2019-03-31 21:59:57+00:00,2.8,31-Mar-2019 (21-59-57),31


Fine
-----
Dati puntuali Lavastoviglie EB1 2019.csv
